# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [ ]:
import json

In [ ]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [ ]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [ ]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [ ]:
print('notice that the instructor and code are tuples now')
courses[1246]

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

# Meng-Ku Chen
## Section 5 (a-c)

In [ ]:
facultyCOSI = set()
for course in courses:
    if course["subject"] == "COSI":
        facultyCOSI.add(course["instructor"][0] + course["instructor"][1])
len(facultyCOSI)

    

In [ ]:
count = 0
for course in courses:
    if course["subject"] == "COSI":
        count += course["enrolled"]
count

In [ ]:
COSI_course = []
res = 0
for course in courses:
    if course["subject"] == "COSI" and course["enrolled"] >= 10:
        COSI_course.append(course["enrolled"])
COSI_course.sort()
middle = len(COSI_course) // 2
if len(COSI_course) % 2 == 0:
    res = (COSI_course[middle] + COSI_course[middle - 1]) / 2
else:
    res = COSI_course[middle]
res